In [1]:
import openai
import pandas as pd
import numpy as np
import os
import json
import random

In [2]:
os.environ["OPENAI_API_KEY"] = "your-key" #can be found in notion 

In [3]:
openai.organization = "instuitaution" 
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
MODEL = "gpt-4o"
#can also try other models as well 

In [6]:
samples = [
    
    passage 

    """
    Yes, I like speaking English.
Uh, because English can use all over the world.  You can use it speak to anyone.
Uh, in Japan---I just speak English in my English class.
Uh, it's about the English book and the topic is in that book.
About the school life.
Or my hobby.
Yes, I want more chance to speak. 
What, uh, like more, uh, sports.
Or games.
Yes, I like that.
Uh, I watch English movies.
Speaking?
Yes, it's, uh, uh, mostly is develop my speaking.
And at the same time, it can develop my speaking - my listening.
Yes.
Uh, one-to-one is better. 
Uh, because it's easy to conversation.  In a group, you don't know when to speak.
Its depends on what you want to do.  If you just want do some research, the most important is reading and if you want to live, uh, foreign - live like America, you should, uh, improve your speaking and listening.

    """ 
    
    """
    
Yes, I like speaking in English.
What?
Yeah, I - you know, I was - I was born in China and I like to - I like watch movie and - in my XXX  school and in high school and in college. 
So, I want to speak English and I think speaking English will be always very good, you know--- 
Yeah, you know, I - since I - I went to Japan, I didn't have any much chance to speak English because the Japanese don't speak in English.
They don't understand what I'm saying so - a little bit about last time, yeah last time a week.
Uh, you know, I was doing part-time job here.  I was working in a hotel.
Yeah, yeah.  There was many foreign customers---
And when the - the Japanese staff don't speak English so I was good to here to - to solve the problems.
Of course, I want to protect English because when - since I went to Japan, I don't have much chance to speak English, so I think my - my English is very big problem here.
Yeah, yeah, yeah.  I don't - I don't remember much of the language so, you know, when - when I  trying to say something, I - I have to think for a moment.  You know, when I'm in China I don't have to hesitate.
I can speak so directly.
For example?
For example what type?
Yeah, I - yeah that matter I - there was any chance to speak.  You know, I was watching the tape, the DVD .
XXX  very cool, there are the many lectures speaking here, and when I was here in, eh, college field, I - I want to say something about some - in college here, about lecture, about something like my - something about what I'm standing.
Yeah, I - you know in China we have to do some test here and one, you know, I don't like some tests already - I don't like some tests, but I have to pass the test, so when you are ready for the test, you have to remember many of the XXX  verbs, grammar and listening and writing and frankly, to be honest, I hated this but - but when you ready for that, when you're practicing for that, you - you'll really get directly here, and I like watch - watch movie here - watch more in American and British, anyway.  I want - I want to speak like, talk like what is it there for.  Yeah.
Yeah, it doesn't matter.
Yeah, I think they are all important but most important is speaking.
Because speaking is the most - most, how to say, uh, it's most way to - to communicate with each other.
Yeah, when, yeah, you can writing here, but if the people don't have chance to read it, what - what do you write, right?  Speaking with each other and the way is the first way to get your point.
    
    """
    
    """
Uh, I think a 100 points is a full points maybe.  I think that I have - I maybe have 70 or 75 points.
No, I - no.
Um, I think this, uh, starting a new - a new thing I think, this will take a little time, uh, maybe for a month, 2 months, or maybe half an year, but finally you - you will not feel nervous about this.
Because, uh, when - when you start something, people always - all of the people will feel nervous I think.
So, just develop your English speaking skills and you will feel confident about.
I think part-time jobs because I - I am now a student and I - I have no part-time job experience.
Uh, I think role play.  This - uh, because I can have some communications with the teachers and this - this picture - storytelling, I think this is a bit - a little familiar with the TOEFL test, uh, speaking test.             
     """
   
]

In [14]:
system_instruction = '''
You are a linguist expert specializing in doing text annotation in the English second language. You will be tasked with making annoations to a given dialogue texts based on some linguistics aspects to compare grammatical features in machine learning models for cross-lingual tasks.

The given text are samples in the dialogue passage from second lanauge speakers of English.  

Make sure to keep the annoation format without any change in passage when giving the annotation output.

A task may ask for one or multiple annotations. Each annoation should be an object with 3 fields: 
type: the type of annoation
annotation_sentence: the annotated sentence
annotation_tokens: all the tokens that are annotated
rationale: the reason why you give the annoation 
grammar correctness: the annotated grammar feature is aligned with the native English speaker's grammar usage

Please return a json object which consists of one or multiple modifications.
'''

In [15]:
def format_prompt(system_instruction, instruction, input):
    # print(input)
    text = input
    # print(text)
    # print(label)
    message = instruction.format(text = text)
    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": message},
    ]
    return messages

In [9]:
def request(samples, prompt_type): # paremeters 
    for sample in samples:
        messages = format_prompt(system_instruction, prompt_type, sample)
        response = openai.chat.completions.create(
            model=MODEL,
            messages= messages,
            temperature=0,  
        )
        ans_model = response.choices[0].message.content
        print(sample)
        print(ans_model)
        print('===================')

Alternative requests and helper functions for special cases (geographical information of the dialogues native background) 


In [10]:
regions = ['China', 'Japan', 'Thailand', 'Eastern Europe',
           'East Asia', 'South Asia', 'Australia and New Zealand', 'Melanesia, Micronesia, and Polynesia'] 
# modify based on the native infornation from ICNALE dataset

In [11]:
def format_prompt_loc(system_instruction, instruction, input):
    # print(input)
    text = input
    # print(text)
    # print(label)
    loc = random.sample(regions,1)
    message = instruction.format(text = text, loc = loc)
    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": message},
    ]
    return messages

### Anaphora and Cataphora

In [18]:
referenve_word_instruction = """
TASK: Read the provided dialogue passage. 
Find all the sentences with reference word and annotate the all the Reference word in the passage. 

Example:

Passage: 
Uh, because English can use all over the world.  You can use it speak to anyone.
Uh, in Japan---I just speak English in my English class.
Uh, it's about the English book and the topic is in that book.
About the school life.

Sentence: 'Yes, I like speaking English.', 'Uh, in Japan---I just speak English in my English class.', 'Uh, it's about the English book and the topic is in that book.'
Annoated Text: 'I','I','it'. 

grammar correctness: True 
{text}

"""

### Noun & verb collocations

In [20]:
noun_verb_instruction = """
TASK: Read the provided dialogue passage. 
Find the sentence in the passage that have Noun & verb collection and annotate all the tokens including Nouns and verbs under the collocations. 

Example:

Passage: 
Uh, because English can use all over the world.  You can use it speak to anyone.
Uh, in Japan---I just speak English in my English class.
Uh, it's about the English book and the topic is in that book.
About the school life.

Sentence: 'Yes, I like speaking English.', 'Uh, in Japan---I just speak English in my English class.', 'Uh, it's about the English book and the topic is in that book.'

Annoated Text: 'speaking English','speak English'. 

grammar correctness: True 


{text}
""" 


### Numbers agreement 

In [22]:
number_agreement_instruction = """
TASK: Read the provided dialogue passage. 
Find the sentence in the passage that have Numbers agreement and annotate all the Numbers agreement.

Example:

Passage: 
Uh, because English can use all over the world.  You can use it speak to anyone.
Uh, in Japan---I just speak English in my English class.
Uh, it's about the English book and the topic is in that book.
About the school life.
Or my hobby.
Yes, I want more chance to speak. 
What, uh, like more, uh, sports.
Or games.
Yes, I like that.
Uh, I watch English movies.
Speaking?
Yes, it's, uh, uh, mostly is develop my speaking.
And at the same time, it can develop my speaking - my listening.
Yes.
Uh, one-to-one is better. 
Uh, because it's easy to conversation.  In a group, you don't know when to speak.
Its depends on what you want to do.  If you just want do some research, the most important is reading and if you want to live, uh, foreign - live like America, you should, uh, improve your speaking and listening.


Sentence: 'What, uh, like more, uh, sports.','Uh, I watch English movies.'
Annoated Text: 'sports','movies'. 

grammar correctness: wrong 


{text}
"""

### Tense agreement 

In [24]:
tense_agreement_instruction = '''
TASK: Read the provided dialogue passage. 
Find the sentence in the passage that have Tense agreement and annotate all the Tense agreement tokens. 

Example:

Passage: 
Uh, because English can use all over the world.  You can use it speak to anyone.
Uh, in Japan---I just speak English in my English class.
Uh, it's about the English book and the topic is in that book.
About the school life.
Or my hobby.
Yes, I want more chance to speak. 
What, uh, like more, uh, sports.
Or games.
Yes, I like that.
Uh, I watch English movies.
Speaking?
Yes, it's, uh, uh, mostly is develop my speaking.
And at the same time, it can develop my speaking - my listening.
Yes.
Uh, one-to-one is better. 
Uh, because it's easy to conversation.  In a group, you don't know when to speak.
Its depends on what you want to do.  If you just want do some research, the most important is reading and if you want to live, uh, foreign - live like America, you should, uh, improve your speaking and listening.


Sentence: 'Uh, I watch English movies.'Yes, it's, uh, uh, mostly is develop my speaking.'
Annoated Text: 'watch','is develop my speaking'. 
grammar correctness: wrong 


{text}
"""

### Subject-Verb agreement

In [26]:
subject_verb_instruction = '''
TASK: Read the provided dialogue passage. 
Find the sentence in the passage that have Subject-Verb agreement and annotate all the Subject-Verb agreement tokens. 

Example:

Passage: 
Uh, because English can use all over the world.  You can use it speak to anyone.
Uh, in Japan---I just speak English in my English class.
Uh, it's about the English book and the topic is in that book.
About the school life.
Or my hobby.
Yes, I want more chance to speak. 
What, uh, like more, uh, sports.
Or games.
Yes, I like that.
Uh, I watch English movies.
Speaking?
Yes, it's, uh, uh, mostly is develop my speaking.
And at the same time, it can develop my speaking - my listening.
Yes.
Uh, one-to-one is better. 
Uh, because it's easy to conversation.  In a group, you don't know when to speak.
Its depends on what you want to do.  If you just want do some research, the most important is reading and if you want to live, uh, foreign - live like America, you should, uh, improve your speaking and listening.


Sentence: 'Uh, I watch English movies.'Yes, it's, uh, uh, mostly is develop my speaking.','In a group, you don't know when to speak.','Its depends on what you want to do. '
Annoated Text: 'I watch','is develop my speaking','you don't know ','Its depends'
grammar correctness: wrong 

{text}
'''

### Speech Acts

In [ ]:


speech_acts_instruction = '''
TASK: Read the provided dialogue passage. 
Find the sentence in the passage that have speech acts and annotate all the speech acts tokens. 

Example:

Passage: 
Uh, because English can use all over the world.  You can use it speak to anyone.
Uh, in Japan---I just speak English in my English class.
Uh, it's about the English book and the topic is in that book.
About the school life.
Or my hobby.
Yes, I want more chance to speak. 
What, uh, like more, uh, sports.
Or games.
Yes, I like that.
Uh, I watch English movies.
Speaking?
Yes, it's, uh, uh, mostly is develop my speaking.
And at the same time, it can develop my speaking - my listening.
Yes.
Uh, one-to-one is better. 
Uh, because it's easy to conversation.  In a group, you don't know when to speak.
Its depends on what you want to do.  If you just want do some research, the most important is reading and if you want to live, uh, foreign - live like America, you should, uh, improve your speaking and listening.


Sentence: ' You can use it speak to anyone.'
Annoated Text: ' You can use it speak to anyone.'
grammar correctness：wrong, and not fit into the context 

{text}
'''

In [ ]:
### Modal Verbs and Expressions

modal_verbs_instruction = '''
TASK: Read the provided dialogue passage. 
Find the sentence in the passage that have speech acts and annotate all the model verbs tokens. 

Example:

Passage: 
Uh, because English can use all over the world.  You can use it speak to anyone.
Uh, in Japan---I just speak English in my English class.
Uh, it's about the English book and the topic is in that book.
About the school life.
Or my hobby.
Yes, I want more chance to speak. 
What, uh, like more, uh, sports.
Or games.
Yes, I like that.
Uh, I watch English movies.
Speaking?
Yes, it's, uh, uh, mostly is develop my speaking.
And at the same time, it can develop my speaking - my listening.
Yes.
Uh, one-to-one is better. 
Uh, because it's easy to conversation.  In a group, you don't know when to speak.
Its depends on what you want to do.  If you just want do some research, the most important is reading and if you want to live, uh, foreign - live like America, you should, uh, improve your speaking and listening.


Sentence: ' You can use it speak to anyone.', 'And at the same time, it can develop my speaking - my listening.'
Annoated Text: 'can','can'
grammar correctness: true

{text}
'''
Please learn the following contents.
The speech act of apology may contain the following functional elements:
APOLOGISING: the element that indicates the act of apologising
REASON: the offense or the reason for the apology
APOLOGISER：the person who apologies
APOLOGISEE: the person to whom the apology is made
INTENSIFIER: the element that upgrades the degree of apology
Here are some examples:
Question: Can you annotate the speech act of apology in the utterance "Ah, I ‘m really
sorry for all that."?
Answer: The annotated version is: Ah, <APOLOGISER> I </APOLOGISER> ‘m
<INTENSIFIER> really </INTENSIFIER> <APOLOGISING> sorry
</APOLOGISING> <REASON> for all that </REASON>.
Question: Can you annotate the speech act of apology in the utterance "Sorry about that,
but I ‘ve got to go to work."?
Answer: The annotated version is: <APOLOGISING> Sorry </APOLOGISING>
<REASON> about that </REASON>, but I ‘ve got to go to work.
Question: Can you annotate the speech act of apology in the utterance "Hello Mr
[gap:name], I ‘m sorry to bother you, my name is Kathy and I represent"?
Answer: The annotated version is: Hello <APOLOGISEE> Mr [gap:name]
</APOLOGISEE>, <APOLOGISER> I </APOLOGISER> ‘m <APOLOGISING>
sorry </APOLOGISING> <REASON> to bother you </REASON>, my name is Kathy
and I represent
Question: Can you annotate the speech act of apology in the utterance "Sorry sorry Mr
[gap:name], I moved too quickly for you."?
Answer: The annotated version is: <APOLOGISING> Sorry </APOLOGISING>
<APOLOGISING> sorry </APOLOGISING> <APOLOGISEE> Mr [gap:name]
</APOLOGISEE>, <REASON> I moved too quickly for you </REASON> 

In [ ]:
### Quantifiers and Numerals


modal_verbs_instruction = '''
TASK: Read the provided dialogue passage. 
Find the sentence in the passage that have Quantifiers and Numerals and annotate all the Quantifiers and Numerals tokens. 

Example:

Passage: 
Yes, I like speaking in English.
What?
Yeah, I - you know, I was - I was born in China and I like to - I like watch movie and - in my XXX  school and in high school and in college. 
So, I want to speak English and I think speaking English will be always very good, you know--- 
Yeah, you know, I - since I - I went to Japan, I didn't have any much chance to speak English because the Japanese don't speak in English.
They don't understand what I'm saying so - a little bit about last time, yeah last time a week.
Uh, you know, I was doing part-time job here.  I was working in a hotel.
Yeah, yeah.  There was many foreign customers---
And when the - the Japanese staff don't speak English so I was good to here to - to solve the problems.
Of course, I want to protect English because when - since I went to Japan, I don't have much chance to speak English, so I think my - my English is very big problem here.
Yeah, yeah, yeah.  I don't - I don't remember much of the language so, you know, when - when I  trying to say something, I - I have to think for a moment.  You know, when I'm in China I don't have to hesitate.
I can speak so directly.
For example?
For example what type?
Yeah, I - yeah that matter I - there was any chance to speak.  You know, I was watching the tape, the DVD .
XXX  very cool, there are the many lectures speaking here, and when I was here in, eh, college field, I - I want to say something about some - in college here, about lecture, about something like my - something about what I'm standing.
Yeah, I - you know in China we have to do some test here and one, you know, I don't like some tests already - I don't like some tests, but I have to pass the test, so when you are ready for the test, you have to remember many of the XXX  verbs, grammar and listening and writing and frankly, to be honest, I hated this but - but when you ready for that, when you're practicing for that, you - you'll really get directly here, and I like watch - watch movie here - watch more in American and British, anyway.  I want - I want to speak like, talk like what is it there for.  Yeah.
Yeah, it doesn't matter.
Yeah, I think they are all important but most important is speaking.
Because speaking is the most - most, how to say, uh, it's most way to - to communicate with each other.
Yeah, when, yeah, you can writing here, but if the people don't have chance to read it, what - what do you write, right?  Speaking with each other and the way is the first way to get your point.
 
Sentence: 'Yeah, you know, I - since I - I went to Japan, I didn't have any much chance to speak English because the Japanese don't speak in English.',
'They don't understand what I'm saying so - a little bit about last time, yeah last time a week.',
'Yeah, I - yeah that matter I - there was any chance to speak. ',
'Yeah, I think they are all important but most important is speaking.'

Annotated Text: ‘any much’,'a little bit','any','all','most'
grammar correctness: true 

{text}
'''
